In [ ]:
from database_wrapper import DatabaseWrapper
from sreality_scraper.sreality.spiders.sreality_spider import SrealityUrlBuilder
import pandas as pd
import numpy as np

db = DatabaseWrapper("listings.db")
df = db.get_df()
db.close_conn()
df

In [ ]:
# set id as index
df.set_index('id', inplace=True)

In [ ]:
# print unique disposition values
print(df["disposition"].unique())
dispositions = df["disposition"].unique()

In [ ]:
# for each unique disposition value try running the map_category_sub_cb from SrealityUrlBuilder
for disposition in dispositions:
    if not disposition or isinstance(disposition, str):
        continue

    if isinstance(disposition, int):    
        df['disposition'] = df['disposition'].replace(disposition, SrealityUrlBuilder.map_category_sub_cb(disposition))


        

In [ ]:
df['disposition'].unique()

In [ ]:
df['disposition'] = df['disposition'].replace('Garsoniéra', '1+kk') # bezrealitky specific
df['disposition'] = df['disposition'].replace('Ostatní', 'other') # bezrealitky specific
df['disposition'] = df['disposition'].replace('atypicky', 'other') # sreality specific
df['disposition'] = df['disposition'].replace('pokoj', 'other') # sreality specific
df['disposition'] = df['disposition'].replace('6+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('6+1', '6-a-více')
df['disposition'] = df['disposition'].replace('7+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('7+1', '6-a-více')


In [ ]:
df['disposition'].sort_values().unique()

In [ ]:
df

In [ ]:
# drop entries with area bigger than 1000
df = df[df['area'] < 1000]
df['area'].sort_values()


In [ ]:
df.loc[:, 'available_from'] = df['available_from'].str.replace(' ', '')


In [ ]:
# Get the indices of rows with invalid date format

invalid_indices = pd.to_datetime(df['available_from'], errors='coerce').isnull()
# get valid indices

df.loc[~invalid_indices, 'available_from'] = pd.to_datetime(df.loc[~invalid_indices, 'available_from'], format='%d.%m.%Y', errors='coerce')
df.loc[invalid_indices, 'available_from'].unique()



In [ ]:
df.loc[df['available_from'] == 'Ihned', 'available_from'] = pd.to_datetime(df.loc[df['available_from'] == 'Ihned', 'updated'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')


In [ ]:
# verify available_from column is in datetime format
df['available_from'] = pd.to_datetime(df['available_from'], errors='coerce')
df['available_from']

In [ ]:
df

In [ ]:
df['balcony'].unique()

In [ ]:
df['balcony'] = df['balcony'].replace('Balk.*', 1, regex=True) # TODO: fix this, the replacement must be ran only on strings
df['balcony'] = df['balcony'].replace(np.nan, 0)
df['balcony'].unique()

In [ ]:
df

In [ ]:
df['cellar'] = df['cellar'].replace('Sklep.*', 1, regex=True)
df['cellar'] = df['cellar'].replace(np.nan, 0)
df

In [ ]:
df['elevator'].unique()

In [ ]:
df[df['elevator'] == 2]

In [ ]:
df[df['elevator'] == 'Výtah']

In [ ]:
df.loc[:, 'elevator'] = df['elevator'].replace('Výtah.*', 1, regex=True)
df.loc[:, 'elevator'] = df['elevator'].replace(np.nan, 0)
df.loc[:, 'elevator'] = df['elevator'].replace(2, 0)
df.loc[:, 'elevator'].unique()

In [ ]:
df.loc[:, 'floor'].sort_values().unique()
df.loc[:, 'floor'] = df['floor'].replace('. podlaží.*', '', regex=True)
df.loc[:, 'floor'] = df['floor'].replace(' z celkem.*', '', regex=True)
df.loc[:, 'floor'] = df['floor'].replace('přízemí', '0', regex=True).astype(int, errors='ignore')
df.loc[:, 'floor'].sort_values().unique()

In [ ]:
df['front_garden'].unique()

In [ ]:
df.loc[:, 'front_garden'] = df['front_garden'].replace('Předzahrádka .*', 1, regex=True)
df.loc[:, 'front_garden'].unique()

In [ ]:
df['furnished'].unique()

In [ ]:
df.loc[:, 'furnished'] = df['furnished'].replace('Nevybaveno', 'nevybaveny').replace('Částečně', 'castecne').replace('Vybaveno', 'vybaveny')
df.loc[:, 'furnished'] = df['furnished'].apply(lambda x: SrealityUrlBuilder.map_furnished_category(x) if isinstance(x, int) else x)



In [ ]:
df['furnished'].unique()

In [ ]:
df.garage.unique()

In [ ]:
df.loc[:, 'garage'] = df['garage'].replace('Garáž.*', 1, regex=True)
df.garage.unique()

In [ ]:
df.loc[:, 'loggie'] = df['garage'].replace('Lodžie.*', 1, regex=True)
df.loggie.unique()

In [ ]:
df.columns

In [ ]:
df.loc[:, 'parking'] = df['parking'].replace('Parkování.*', 1, regex=True)
df.parking.unique()


In [ ]:
df.loc[:, 'pets'] = df['pets'].replace('Domácí mazlíčci vítáni', 1, regex=True)
df.pets.unique()

In [ ]:
# drop public_transport column
df.drop(columns=['public_transport'], inplace=True)

In [ ]:
df.columns

In [ ]:
df.rent.unique()


In [ ]:
df.loc[:, 'rent'] = df['rent'].apply(lambda x: int(x.replace(' ', '').replace('Kč', '').replace('€', '')) if isinstance(x, str) else x)

df['rent'].sort_values().unique()

In [ ]:
# 'security_deposit'

# print unique security_deposit values for rows wher url contains sreality
df.loc[df['url'].str.contains('bezrealitky')]['security_deposit'].unique()

In [ ]:
# drop security_deposit column
df.drop(columns=['security_deposit'], inplace=True)

In [ ]:
# 'service_fees', 'status', 'terrace', 'type', 'updated', 'url'

# unique values for service_fees
df.service_fees

In [ ]:
df.drop(columns=['service_fees'], inplace=True)

In [ ]:
# 'status'

# unique values for status
df.status.unique()

In [ ]:
# 'terrace', 
df.loc[:, 'terrace'] = df['terrace'].apply(lambda x: 1 if isinstance(x, str) and "Terasa" in x else x)
df.loc[:, 'terrace'].unique()

In [ ]:
# 'type'
df.type.unique()
df.loc[:, 'type'] = df['type'].apply(lambda x: x.replace('Cihla', 'cihlova').replace('Panel', 'panelova').replace('Smíšená', 'ostatni').replace('Skeletová', 'ostatni').replace('Nízkoenergetická', 'ostatni').replace('Montovaná', 'ostatni').replace('Dřevostavba', 'ostatni').replace('Kamenná', 'ostatni').replace('Ostatní', 'ostatni') if isinstance(x, str) else x)
df.loc[:, 'type'] = df['type'].apply(lambda x: SrealityUrlBuilder.map_building_type(x) if isinstance(x, int) else x)
df.type.unique()

In [ ]:
# save df to filtered a sqlite database without using the databasewrapper
import sqlite3
conn = sqlite3.connect('filtered_listings.db')
df.to_sql('listings', conn, if_exists='replace')
conn.close()